In [ ]:
from generate import *

In [ ]:
!export PATH="/share/project/quansun/evaclip_running/env/evaclip/bin:$PATH"

In [ ]:
from transformers import DataCollatorForLanguageModeling
import transformers

IGNORE_INDEX = -100
DEFAULT_PAD_TOKEN = "[PAD]"
DEFAULT_EOS_TOKEN = "</s>"
DEFAULT_BOS_TOKEN = "</s>"
DEFAULT_UNK_TOKEN = "</s>"
PROMPT_DICT = {
    "prompt_input": (
        "Below is an instruction that describes a task, paired with an input that provides further context. "
        "Write a response that appropriately completes the request.\n\n"
        "### Instruction:\n{instruction}\n\n### Input:\n{input}\n\n### Response:"
    ),
    "prompt_no_input": (
        "Below is an instruction that describes a task. "
        "Write a response that appropriately completes the request.\n\n"
        "### Instruction:\n{instruction}\n\n### Response:"
    ),
}


def prepare_pile_dataset():
    base_url = "/share/project/quansun/Pile/"
    data_files = {
        "train": [base_url + "train/" + f"{idx:02d}.jsonl.zst" for idx in range(30)],
        "validation": base_url + "val.jsonl.zst",
        "test": base_url + "test.jsonl.zst",
    }
    pile_dataset = load_dataset("json", data_files=data_files, streaming=True)
    # return pile_dataset['train'], pile_dataset['validation'], pile_dataset['test']
    return pile_dataset

def make_pile_data_module(tokenizer: transformers.PreTrainedTokenizer):
    """Make dataset and collator for Pile fine-tuning."""
    raw_datasets = prepare_pile_dataset()
    data_collator = DataCollatorForLanguageModeling(tokenizer, mlm=False)

    def tokenize(example):
        outputs = tokenizer(
            example["text"],
            return_tensors="pt",
            padding="longest",
            max_length=tokenizer.model_max_length,
            truncation=True,
        )
        # outputs['labels'] = copy.deepcopy(outputs['input_ids'])
        return outputs

    tokenized_datasets = raw_datasets.map(
        tokenize, batched=True, remove_columns=raw_datasets["train"].column_names
    )

    return dict(train_dataset=tokenized_datasets["train"], eval_dataset=tokenized_datasets["validation"], data_collator=data_collator)

In [ ]:
# model = transformers.AutoModelForCausalLM.from_pretrained(
#         model_args.model_name_or_path,
#         cache_dir=training_args.cache_dir,
#     )

tokenizer = transformers.AutoTokenizer.from_pretrained(
    "/share/project/qiying/model_cache/LLaMA/hf/llama-7b",
    cache_dir="/share/project/qiying/model_cache/LLaMA/hf/llama-7b",
    model_max_length=1024,
    padding_side="right",
    use_fast=False,
)